In [42]:
import cv2
import numpy as np

# Read image
im = cv2.imread("(2.1).jpeg",cv2.IMREAD_REDUCED_COLOR_4)

# Set up the parameters for blob detection
params = cv2.SimpleBlobDetector_Params()
params.filterByArea = True
params.minArea = 100
params.maxArea = 100000
params.filterByCircularity = True
params.minCircularity = 0.1
params.filterByConvexity = True
params.minConvexity = 0.5
params.filterByInertia = True
params.minInertiaRatio = 0.5


# Create the detector with the specified parameters
detector = cv2.SimpleBlobDetector_create(params)

# Detect blobs
keypoints = detector.detect(im)

# Draw detected blobs as red circles
# cv2.DRAW_MATCHES_FLAGS_DRAW_RICH_KEYPOINTS ensures the size of the circle corresponds to the size of blob
im_with_keypoints = cv2.drawKeypoints(im, keypoints, np.array([]), (0, 0, 255), cv2.DRAW_MATCHES_FLAGS_DRAW_RICH_KEYPOINTS)

# Show keypoints
cv2.namedWindow("Keypoints", cv2.WINDOW_NORMAL)  # Create a resizable window
cv2.resizeWindow("Keypoints", 800, 600)  # Set the window size (adjust as needed)
cv2.imshow("Keypoints", im_with_keypoints)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [6]:
import cv2
import numpy as np
from skimage.feature import blob_log
from skimage.color import rgb2gray

# Read the image

im = cv2.imread('(2.1).jpeg',cv2.IMREAD_REDUCED_COLOR_4)

# Convert the image to grayscale
im_gray = cv2.cvtColor(im, cv2.COLOR_BGR2GRAY)

# Apply Gaussian blur to reduce noise
#im_blur = cv2.GaussianBlur(im_gray, (0, 0), 1.5)

# Perform blob detection using the Laplacian of Gaussians (LoG)
blobs = blob_log(im_gray, min_sigma=1, max_sigma=30, threshold=0.1)

# Draw circles on the original image for detected blobs
if blobs is not None:
    for blob in blobs:
        y, x, r = blob
        cv2.circle(im, (int(x), int(y)), int(r), (0, 0, 255), 1)  # Draw a red circle


# Display the image with detected circles
cv2.namedWindow("Blob Detection", cv2.WINDOW_NORMAL)  # Create a resizable window
cv2.resizeWindow("Blob Detection", 800, 600)  # Set the window size (adjust as needed)
cv2.imshow("Blob Detection", im)
cv2.waitKey(0)
cv2.destroyAllWindows()


